In [1]:
import pandas as pd
import numpy as np
import random

def generate_chaotic_data(num_records=100):
    # Some sample names with mixed case and possible None values
    names = ["Alice", "bob", "CHARLIE", "David", None, "Eva", "frank", "Gina", "henry", "Isaac"]
    ages = [25, "thirty", 35, None, 45, 50, "forty", 29, None, 40]
    heights = [165, 170, 180, 175, None, 160, 172, 168, 174, "one seventy"]
    weights = [55, 65, None, 70, 68, 60, "sixty five", 58, 62, 66]
    emails = [
        "alice@example.com", "bob[at]email.com", "charlie.email.com", "david@mail.com",
        "eva@example", None, "frank@mail.com", "gina@email.com", "henry@mail", "isaac@mail.com"
    ]
    join_dates = [
        "2023-01-10", "2023/02/20", "March 15, 2023", "2023-04-01", None, "2023-06-30",
        "2023-07-15", "2023-08-20", "2023-09-25", "2023-10-10"
    ]

    data = {
        "Name": [],
        "Age": [],
        "Height_cm": [],
        "Weight_kg": [],
        "Email": [],
        "JoinDate": []
    }

    for _ in range(num_records):
        i = random.randint(0, 9)
        # Add some randomness/noise
        name = names[i]
        # Add noise to name sometimes
        if random.random() < 0.1:
            name = name.lower() if name else name
        age = ages[i]
        height = heights[i]
        weight = weights[i]
        email = emails[i]
        join_date = join_dates[i]

        # Occasionally insert extra noise
        if random.random() < 0.05:
            age = "unknown"
        if random.random() < 0.05:
            weight = "N/A"
        if random.random() < 0.05:
            join_date = "unknown"

        data["Name"].append(name)
        data["Age"].append(age)
        data["Height_cm"].append(height)
        data["Weight_kg"].append(weight)
        data["Email"].append(email)
        data["JoinDate"].append(join_date)

    df = pd.DataFrame(data)
    return df

def save_messy_data(filepath="E:/da_dataset.csv"):
    df = generate_chaotic_data(100)
    df.to_csv(filepath, index=False)
    print(f"Messy data saved to {filepath}")

if __name__ == "__main__":
    save_messy_data()


Messy data saved to E:/da_dataset.csv


In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from collections import Counter

# Step 1: Normalize Text (names, emails)
def normalize_text(df):
    df['Name'] = df['Name'].astype(str).str.strip().str.title()

    def clean_email(email):
        if pd.isna(email) or email.lower() in ['none', 'nan', '']:
            return np.nan
        email = email.replace("[at]", "@").strip()
        pattern = r"[^@]+@[^@]+\.[^@]+"
        if re.match(pattern, email):
            return email.lower()
        return np.nan
    df['Email'] = df['Email'].apply(clean_email)
    return df

# Step 2: Fix numeric columns (convert text numbers to numerics)
def fix_numeric_columns(df):
    text_to_num = {
        "thirty": 30, "forty": 40, "sixty five": 65, "one seventy": 170,
        "unknown": np.nan, "n/a": np.nan
    }
    for col in ['Age', 'Height_cm', 'Weight_kg']:
        def convert_value(val):
            if pd.isna(val):
                return np.nan
            if isinstance(val, (int, float)):
                return val
            val = str(val).lower()
            if val in text_to_num:
                return text_to_num[val]
            try:
                return float(val)
            except:
                return np.nan
        df[col] = df[col].apply(convert_value)
    return df

# Step 3: Impute missing values intelligently
def impute_missing(df):
    numeric_cols = ['Age', 'Height_cm', 'Weight_kg']
    imputer = SimpleImputer(strategy='median')
    df[numeric_cols] = imputer.fit_transform(df[numeric_cols])

    df.dropna(subset=['Name', 'Email'], inplace=True)

    df['JoinDate'] = pd.to_datetime(df['JoinDate'], errors='coerce')
    earliest_date = df['JoinDate'].min()
    df['JoinDate'] = df['JoinDate'].fillna(earliest_date)
    return df

# Step 4: Remove outliers using z-score
def remove_outliers(df):
    numeric_cols = ['Age', 'Height_cm', 'Weight_kg']
    scaler = StandardScaler()
    scaled = scaler.fit_transform(df[numeric_cols])
    z_scores = np.abs(scaled)
    filter_mask = (z_scores < 3).all(axis=1)
    removed_count = len(df) - np.sum(filter_mask)
    df_clean = df[filter_mask].copy()
    print(f"Removed {removed_count} outlier rows.")
    return df_clean

# Step 5: Summary report
def summary_report(df_raw, df_clean):
    print("\n=== Chaos to Clarity Cleaning Report ===")
    print(f"Original rows: {len(df_raw)}")
    print(f"Cleaned rows: {len(df_clean)}")
    print("\nColumn Data Types:")
    print(df_clean.dtypes)
    print("\nSample cleaned data:")
    print(df_clean.head())
    print("\nUnique Names:")
    print(sorted(df_clean['Name'].unique()))
    print("\nMost common email domains:")
    domains = df_clean['Email'].str.split('@').str[1]
    print(Counter(domains).most_common(5))

def main():
    filepath = "E:/da_dataset.csv"
    print(f"Loading messy data from {filepath} ...")
    df_raw = pd.read_csv(filepath)

    df_norm = normalize_text(df_raw)
    df_num_fixed = fix_numeric_columns(df_norm)
    df_imputed = impute_missing(df_num_fixed)
    df_clean = remove_outliers(df_imputed)

    summary_report(df_raw, df_clean)

    output_path = "E:/da_dataset_cleaned.csv"
    df_clean.to_csv(output_path, index=False)
    print(f"\nCleaned data saved as {output_path}")

if __name__ == "__main__":
    main()


Loading messy data from E:/da_dataset.csv ...
Removed 0 outlier rows.

=== Chaos to Clarity Cleaning Report ===
Original rows: 64
Cleaned rows: 64

Column Data Types:
Name                 object
Age                 float64
Height_cm           float64
Weight_kg           float64
Email                object
JoinDate     datetime64[ns]
dtype: object

Sample cleaned data:
    Name   Age  Height_cm  Weight_kg              Email   JoinDate
0    Bob  35.0      170.0       65.0      bob@email.com 2023-02-20
1  Alice  25.0      165.0       55.0  alice@example.com 2023-02-20
2  Isaac  35.0      170.0       66.0     isaac@mail.com 2023-02-20
4  Frank  40.0      172.0       65.0     frank@mail.com 2023-02-20
7  Isaac  40.0      170.0       66.0     isaac@mail.com 2023-02-20

Unique Names:
['Alice', 'Bob', 'David', 'Frank', 'Gina', 'Isaac']

Most common email domains:
[('mail.com', 29), ('email.com', 21), ('example.com', 14)]

Cleaned data saved as E:/da_dataset_cleaned.csv
